In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools as it
from collections import defaultdict
import seaborn as sns
pd.set_option('display.max_columns', None)

In [2]:
train_trs = pd.read_csv('/home/user/Desktop/courseProject/data/train_transaction.csv')
train_idn = pd.read_csv('/home/user/Desktop/courseProject/data/train_identity.csv')

In [3]:
data_merged = pd.merge(train_trs, train_idn, left_on='TransactionID', right_on='TransactionID', how='left')

In [4]:
def get_share_of_NaN(df):
    result = pd.DataFrame(columns=['Name', 'Number_of_NaN', 'Share_of_NaN'])
    colcount = df.count()
    length = len(df)
    for col_name in colcount.keys():
        result.loc[len(result)] = [col_name, length-colcount[col_name], (length-colcount[col_name])/length]
    return result

def remove_columns_with_many_NaN(df, max_nan_rate):
    '''
    Параметры:
    df - DataFrame
    max_nan_rate - максимальная допустимая доля NaN в колонках датафрейма
    Функция возвращает:
    1) новый датафрейм, в котором удалены колонки, в которых доля NaN болше, чем max_nan_rate
    2) список удалённых колонок
    '''
    df_copy = df.copy()
    removed_columns = []
    nan_stat = get_share_of_NaN(df)
    for i in range(len(nan_stat)):
        column = nan_stat.loc[i]
        if(column['Share_of_NaN'] > max_nan_rate):
            removed_columns.append(column['Name'])
            
    df_copy.drop(columns=removed_columns, inplace=True)
    return df_copy, removed_columns

def remove_columns_with_big_correlation(df, max_corr):
    '''
    Параметры:
    df - DataFrame
    max_corr - максимальная допустимая корреляция между колонками
    Функция возвращает:
    1) новый датафрейм, в котором удалены колонки, в которых корреляция болше, чем max_corr
    2) множество удалённых колонок
    '''
    df_copy = df.copy()
    removed_columns = set()
    corrs = df.corr()
    cols = corrs.columns
    for i in range(len(cols)):
        col_name_1 = cols[i]
        if col_name_1 in {'TransactionID', 'isFraud', 'TransactionDT'} or col_name_1 in removed_columns:
            continue
        
        for j in range(i+1, len(cols)):
            col_name_2 = cols[j]
            if abs(corrs[col_name_1][col_name_2]) > max_corr:
                removed_columns.add(col_name_2)

    df_copy.drop(columns=removed_columns, inplace=True)
    return df_copy, removed_columns

In [5]:
print(type(train_idn))
print(type(data_merged))
print(len(data_merged.columns))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
434


In [6]:
%%time
data_merged, removed_nan_cols = remove_columns_with_many_NaN(data_merged, 0.85)
data_merged_rm_cols, removed_corr_cols = remove_columns_with_big_correlation(data_merged, 0.9)

In [10]:
data_merged_rm_cols.to_csv('/home/user/Desktop/courseProject/data/train_cols.csv')

In [7]:
print(len(data_merged_rm_cols.columns))

218


In [31]:
cat_features_mask = (data_merged_rm_cols.dtypes == "object").values
val_features_mask = (data_merged.dtypes != "object").values
for i in range(len(data_merged_rm_cols.columns)):
    #print(i)
    #print(data_merged_rm_cols.iloc[:,i].columns[0])
    if data_merged_rm_cols.iloc[:,i].name in ['card1', 'card2', 'card3', 'card5', 'addr1', 'addr2']:
        #print("changed")
        cat_features_mask[i] = True
        val_features_mask[i] = False
#print(cat_features_mask)



    #real_cols = train_trs[train_trs.columns[~cat_features_mask]]
cat_cols = data_merged_rm_cols[data_merged_rm_cols.columns[cat_features_mask]]
var_cols = data_merged_rm_cols[data_merged_rm_cols.columns[~cat_features_mask]]
print(cat_cols)

       ProductCD  card1  card2  card3       card4  card5   card6  addr1  \
0              W  13926    NaN  150.0    discover  142.0  credit  315.0   
1              W   2755  404.0  150.0  mastercard  102.0  credit  325.0   
2              W   4663  490.0  150.0        visa  166.0   debit  330.0   
3              W  18132  567.0  150.0  mastercard  117.0   debit  476.0   
4              H   4497  514.0  150.0  mastercard  102.0  credit  420.0   
...          ...    ...    ...    ...         ...    ...     ...    ...   
590535         W   6550    NaN  150.0        visa  226.0   debit  272.0   
590536         W  10444  225.0  150.0  mastercard  224.0   debit  204.0   
590537         W  12037  595.0  150.0  mastercard  224.0   debit  231.0   
590538         W   7826  481.0  150.0  mastercard  224.0   debit  387.0   
590539         W  15066  170.0  150.0  mastercard  102.0  credit  299.0   

        addr2 P_emaildomain R_emaildomain   M1   M2   M3   M4   M5   M6   M7  \
0        87.0      

In [59]:
# remove all nones
def proc_cat_col(df, cat_features_mask):
    cat_cols = df.columns[cat_features_mask]
    #print(cat_cols)
    for col in cat_cols:
        dc = df.loc[:,col]
        dc = dc.fillna(dc.mode()[0], inplace=True)


def proc_val_col(df, cat_features_mask, sample_size):
    val_cols = df.columns[~cat_features_mask]
    for col in val_cols:
        dc = df.loc[:,col]
        dc = dc.fillna(dc.sample(n=sample_size).dropna().mode()[0], inplace=True)

In [53]:
%%time

data_test = data_merged_rm_cols.copy()
data_test.describe()

CPU times: user 1.88 s, sys: 168 ms, total: 2.04 s
Wall time: 2.04 s


,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,C1,C3,C5,C13,D1,D3,D4,D5,D10,D11,D15,V1,V2,V3,V4,V6,V7,V8,V9,V10,V12,V14,V15,V17,V19,V23,V24,V25,V26,V27,V29,V32,V35,V37,V38,V39,V41,V42,V44,V46,V47,V49,V52,V53,V55,V56,V60,V61,V62,V65,V66,V67,V68,V70,V74,V75,V77,V78,V81,V82,V83,V86,V87,V95,V98,V99,V100,V104,V105,V107,V108,V109,V110,V111,V112,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V129,V130,V131,V135,V136,V169,V170,V171,V172,V173,V174,V175,V176,V180,V181,V184,V185,V186,V187,V188,V189,V191,V194,V195,V199,V200,V204,V205,V207,V208,V209,V210,V214,V215,V217,V220,V221,V223,V224,V226,V227,V228,V229,V230,V234,V238,V240,V241,V242,V246,V247,V248,V250,V252,V255,V258,V260,V261,V262,V264,V267,V268,V270,V274,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V300,V303,V305,V310,V311,V312,V313,V314,V319,V320,id_01,id_02,id_05,id_06,id_11,id_13,id_17,id_19,id_20
count,5.905400e+05,590540.000000,5.905400e+05,590540.000000,590540.000000,581607.000000,588975.000000,586281.000000,524834.000000,524834.000000,238269.000000,590540.000000,590540.000000,590540.000000,590540.000000,589271.000000,327662.000000,421618.000000,280699.000000,514518.000000,311253.000000,501427.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,421571.000000,421571.000000,421571.00000,421571.000000,421571.000000,421571.000000,421571.000000,421571.000000,421571.000000,421571.000000,421571.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,139819.000000,139819.000000,139819.000000,139631.000000,139631.000000,139819.000000,139819.000000,139631.000000,139819.000000,139631.000000,139819.000000,139819.000000,139631.000000,139631.000000,139819.000000,139819.000000,139631.000000,139819.000000,139819.000000,139631.000000,139819.000000,139631.000000,139631.000000,139631.000000,139819.000000,139819.000000,139819.000000,139631.000000,139631.000000,130430.000000,141416.000000,141416.000000,130430.000000,130430.000000,130430.000000,141416.000000,130430.000000,130430.000000,130430.000000,141416.000000,141416.000000,130430.000000,130430.000000,130430.000000,130430.000000,130430.000000,130430.000000,141416.000000,130430.000000,141416.000000,130430.000000,130430.000000,130430.000000,130430.000000,130430.000000,130430.000000,130430.000000,141416.000000,130430.000000,589271.000000,589271.000000,589271.000000,590528.000000,590528.000000,590528.000000,590528.000000,589271.000000,589271.000000,590528.000000,590528.000000,589271.000000,590528.000000,590528.000000,590528.000000,590528.000000,590528.000000,589271.000000,589271.000000,590528.000000,590528.000000,144233.000000,140872.000000,136865.000000,136865.000000,140978.000000,127320.000000,139369.000000,139318.000000,139261.000000
mean,3.282270e+06,0.034990,7.372311e+06,135.027176,9898.734658,362.555488,153.194925,199.278897,290.733794,86.800630,118.502180,14.092458,0.005644,5.571526,32.539918,94.347568,28.343348,140.002441,42.335965,123.982137,146.621465,163.744579,0.999945,1.045204,1.078075,0.846456,1.045686,1.072870,1.027704,1.041529,0.463915,0.559711,0.999500,0.122342,0.134040,0.816371,1.034791,1.05809

In [54]:
%%time
proc_cat_col(data_test, cat_features_mask)
data_test.describe()

CPU times: user 3.73 s, sys: 34.1 ms, total: 3.77 s
Wall time: 3.76 s


,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,C1,C3,C5,C13,D1,D3,D4,D5,D10,D11,D15,V1,V2,V3,V4,V6,V7,V8,V9,V10,V12,V14,V15,V17,V19,V23,V24,V25,V26,V27,V29,V32,V35,V37,V38,V39,V41,V42,V44,V46,V47,V49,V52,V53,V55,V56,V60,V61,V62,V65,V66,V67,V68,V70,V74,V75,V77,V78,V81,V82,V83,V86,V87,V95,V98,V99,V100,V104,V105,V107,V108,V109,V110,V111,V112,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V129,V130,V131,V135,V136,V169,V170,V171,V172,V173,V174,V175,V176,V180,V181,V184,V185,V186,V187,V188,V189,V191,V194,V195,V199,V200,V204,V205,V207,V208,V209,V210,V214,V215,V217,V220,V221,V223,V224,V226,V227,V228,V229,V230,V234,V238,V240,V241,V242,V246,V247,V248,V250,V252,V255,V258,V260,V261,V262,V264,V267,V268,V270,V274,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V300,V303,V305,V310,V311,V312,V313,V314,V319,V320,id_01,id_02,id_05,id_06,id_11,id_13,id_17,id_19,id_20
count,5.905400e+05,590540.000000,5.905400e+05,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,238269.000000,590540.000000,590540.000000,590540.000000,590540.000000,589271.000000,327662.000000,421618.000000,280699.000000,514518.000000,311253.000000,501427.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,421571.000000,421571.000000,421571.00000,421571.000000,421571.000000,421571.000000,421571.000000,421571.000000,421571.000000,421571.000000,421571.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,139819.000000,139819.000000,139819.000000,139631.000000,139631.000000,139819.000000,139819.000000,139631.000000,139819.000000,139631.000000,139819.000000,139819.000000,139631.000000,139631.000000,139819.000000,139819.000000,139631.000000,139819.000000,139819.000000,139631.000000,139819.000000,139631.000000,139631.000000,139631.000000,139819.000000,139819.000000,139819.000000,139631.000000,139631.000000,130430.000000,141416.000000,141416.000000,130430.000000,130430.000000,130430.000000,141416.000000,130430.000000,130430.000000,130430.000000,141416.000000,141416.000000,130430.000000,130430.000000,130430.000000,130430.000000,130430.000000,130430.000000,141416.000000,130430.000000,141416.000000,130430.000000,130430.000000,130430.000000,130430.000000,130430.000000,130430.000000,130430.000000,141416.000000,130430.000000,589271.000000,589271.000000,589271.000000,590528.000000,590528.000000,590528.000000,590528.000000,589271.000000,589271.000000,590528.000000,590528.000000,589271.000000,590528.000000,590528.000000,590528.000000,590528.000000,590528.000000,589271.000000,589271.000000,590528.000000,590528.000000,144233.000000,140872.000000,136865.000000,136865.000000,140978.000000,127320.000000,139369.000000,139318.000000,139261.000000
mean,3.282270e+06,0.034990,7.372311e+06,135.027176,9898.734658,361.926886,153.186458,199.471611,291.653527,86.822813,118.502180,14.092458,0.005644,5.571526,32.539918,94.347568,28.343348,140.002441,42.335965,123.982137,146.621465,163.744579,0.999945,1.045204,1.078075,0.846456,1.045686,1.072870,1.027704,1.041529,0.463915,0.559711,0.999500,0.122342,0.134040,0.816371,1.034791,1.05809

In [60]:
%%time
proc_val_col(data_test, cat_features_mask, 10000)
data_test.describe()

CPU times: user 5.17 s, sys: 35.9 ms, total: 5.21 s
Wall time: 5.22 s


,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,C1,C3,C5,C13,D1,D3,D4,D5,D10,D11,D15,V1,V2,V3,V4,V6,V7,V8,V9,V10,V12,V14,V15,V17,V19,V23,V24,V25,V26,V27,V29,V32,V35,V37,V38,V39,V41,V42,V44,V46,V47,V49,V52,V53,V55,V56,V60,V61,V62,V65,V66,V67,V68,V70,V74,V75,V77,V78,V81,V82,V83,V86,V87,V95,V98,V99,V100,V104,V105,V107,V108,V109,V110,V111,V112,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V129,V130,V131,V135,V136,V169,V170,V171,V172,V173,V174,V175,V176,V180,V181,V184,V185,V186,V187,V188,V189,V191,V194,V195,V199,V200,V204,V205,V207,V208,V209,V210,V214,V215,V217,V220,V221,V223,V224,V226,V227,V228,V229,V230,V234,V238,V240,V241,V242,V246,V247,V248,V250,V252,V255,V258,V260,V261,V262,V264,V267,V268,V270,V274,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V300,V303,V305,V310,V311,V312,V313,V314,V319,V320,id_01,id_02,id_05,id_06,id_11,id_13,id_17,id_19,id_20
count,5.905400e+05,590540.000000,5.905400e+05,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.00000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.00000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.00000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000
mean,3.282270e+06,0.034990,7.372311e+06,135.027176,9898.734658,361.926886,153.186458,199.471611,291.653527,86.822813,47.812842,14.092458,0.005644,5.571526,32.539918,94.144827,15.726349,99.955209,20.123384,108.021541,77.279051,139.035379,0.999971,1.023826,1.041150,0.919072,1.024080,1.038407,1.014602,1.021888,0.244513,0.616429,0.999565,0.106582,0.116773,0.840026,1.030310,1.050613,0.9